# Yelp Data Analysis - Relationship between Density and Quality


In [3]:
import csv
import pandas as pd
import numpy as np
import pandas as pd
import seaborn as sns
import operator #need this for sorting dicts later
import matplotlib.cm as cm #for colormapping later
from matplotlib.colors import LogNorm
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
## import the yelp business dataset as a csv
business_df = pd.read_csv('business.csv')

In [20]:
business_df.head()

,attributes.Ambience.divey,attributes.RestaurantsDelivery,attributes.DogsAllowed,postal_code,hours.Thursday,attributes.HairSpecializesIn.coloring,attributes.BestNights.sunday,attributes.BYOB,attributes.AgesAllowed,attributes.Music.video,...,attributes.Caters,attributes.RestaurantsReservations,attributes.DietaryRestrictions.dairy-free,attributes.DietaryRestrictions.vegan,attributes.Ambience.romantic,attributes.Music.jukebox,attributes.Ambience.upscale,attributes.RestaurantsTakeOut,attributes.BikeParking,attributes.OutdoorSeating
0,NaN,NaN,NaN,44143,10:00-21:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN
1,False,True,NaN,28215,10:00-22:00,NaN,NaN,NaN,NaN,NaN,...,NaN,False,NaN,NaN,False,NaN,False,True,NaN,False
2,NaN,NaN,NaN,M4K 1N7,10:00-19:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False
3,NaN,NaN,NaN,85258,9:00-17:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,False,NaN,85016,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,NaN,NaN,False,NaN,False,True,True,False


## Cleaning the Dataset

We are going to remove the unecessary columns from this dataset:

In [26]:
business_df = business_df[['name','postal_code','latitude','longitude','city','state','neighborhood','categories','stars','review_count']]

In [27]:
business_df.head()

,name,postal_code,latitude,longitude,city,state,neighborhood,categories,stars,review_count
0,Richmond Town Square,44143,41.541716,-81.493116,Richmond Heights,OH,NaN,"[u'Shopping', u'Shopping Centers']",2.0,17
1,South Florida Style Chicken & Ribs,28215,35.236870,-80.741976,Charlotte,NC,Eastland,"[u'Food', u'Soul Food', u'Convenience Stores',...",4.5,4
2,The Tea Emporium,M4K 1N7,43.677126,-79.353285,Toronto,ON,Riverdale,"[u'Food', u'Coffee & Tea']",4.5,7
3,TRUmatch,85258,33.565082,-111.916400,Scottsdale,AZ,NaN,"[u'Professional Services', u'Matchmakers']",3.0,3
4,Blimpie,85016,33.505928,-112.038847,Phoenix,AZ,NaN,"[u'Sandwiches', u'Restaurants']",4.5,10


In [28]:
business_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156639 entries, 0 to 156638
Data columns (total 10 columns):
name            156639 non-null object
postal_code     156086 non-null object
latitude        156638 non-null float64
longitude       156638 non-null float64
city            156636 non-null object
state           156639 non-null object
neighborhood    62236 non-null object
categories      156639 non-null object
stars           156639 non-null float64
review_count    156639 non-null int64
dtypes: float64(3), int64(1), object(6)
memory usage: 12.0+ MB


We can focus on examining businesses only in Toronto

In [49]:
toronto_restaurants_df = business_df[business_df['city'] == 'Toronto']
toronto_restaurants_df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
toronto_restaurants_df.head()

,name,postal_code,latitude,longitude,city,state,neighborhood,categories,stars,review_count
2,The Tea Emporium,M4K 1N7,43.677126,-79.353285,Toronto,ON,Riverdale,"[u'Food', u'Coffee & Tea']",4.5,7
26,Hart House Theatre,M5S 3H3,43.663669,-79.394687,Toronto,ON,University of Toronto,"[u'Arts & Entertainment', u'Performing Arts']",4.0,8
38,McCarthy's Irish Pub,M4L 2B5,43.678049,-79.314774,Toronto,ON,Upper Beach,"[u'Pubs', u'Restaurants', u'Bars', u'Irish', u...",4.0,5
44,Paris Bakery & Pastry,M6H 3Z5,43.662401,-79.444706,Toronto,ON,Wallace Emerson,"[u'Bakeries', u'Food']",3.0,16
53,Oishi Sushi,M3J 2G5,43.763510,-79.490750,Toronto,ON,NaN,"[u'Asian Fusion', u'Restaurants', u'Sushi Bars']",2.0,27


In [52]:
def is_restaurant(s):
    categories = s
    for x in categories:
        if x == "u'Restaurants":
            return "Restaurant"
        else:
            return "Not Restaurant"

toronto_restaurants_df["business_type"] = toronto_restaurants_df[['categories']].apply(is_restaurant,axis=1)  


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [ ]:

convert the categories from list of ojects tolist of strings
create a list with restaurant and not restaurant types